In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [15]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, models

model = models.Sequential()

model.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', data_format='channels_last', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',#optimizers.RMSprop(lr=1e-3),
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 1)       28        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 1)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 12321)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                123220    
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 33        
Total params: 123,281
Trainable params: 123,281
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=20,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )
# model.save('first_try.h5')

Epoch 1/20


TypeError: object of type 'float' has no len()